## Import dataset from locally stored CEPII database

In [6]:
import pandas as pd

df = pd.read_csv('gravity.csv', low_memory=False)

## __Data cleaning procedures:__
- Isolate the columns
- Eliminate all rows for missing data for key variables
- Create columns needed for regression


In [7]:
#Isolating the desired columns
columns = ['year', 'iso3_o', 'iso3_d', 'distw', 'comlang_off', 'comlang_ethno', 'comcol', 'col45','heg_o', 'heg_d', 'col_dep_ever', 'sibling_ever', 'gdp_o', 'gdp_d', 'eu_o', 'eu_d', 'tradeflow_comtrade_o', 'tradeflow_comtrade_d', 'tradeflow_baci', 'tradeflow_imf_o', 'tradeflow_imf_d']
df = df[columns]

In [8]:
#We have that there are 5 different methods of measuring tradeflow, we hence take an average to erradicate the issues of missing values
pd.options.mode.chained_assignment = None
df['tradeflow'] = df[['tradeflow_comtrade_o', 'tradeflow_comtrade_d', 'tradeflow_baci', 'tradeflow_imf_o', 'tradeflow_imf_d']].mean(axis=1)
df = df.drop(['tradeflow_comtrade_o', 'tradeflow_comtrade_d', 'tradeflow_baci', 'tradeflow_imf_o', 'tradeflow_imf_d'], axis=1)

In [10]:
#Drop all the rows where any of the fundamental variables in the gravity equation are missing and impute remaining missing tradeflow data
df = df.drop(df[df['gdp_o'].isna() | df['gdp_d'].isna() | df['distw'].isna()].index)
df = df.drop(df[df['iso3_o'] == df['iso3_d']].index)
df['tradeflow'] = df['tradeflow'].fillna(1)

In [11]:
#Create dummy for origin and destination being EU countries
df['both_eu'] = (df['eu_o'] == 1) & (df['eu_d'] == 1)
df['both_eu'] = df['both_eu'].astype(int)

In [12]:
#Create dummy for either being EU country
df['one_eu'] = (((df['eu_o'] == 1) | (df['eu_d'] == 1)) & (df['both_eu'] == 0))
df['one_eu'] = df['one_eu'].astype(int)

In [13]:
df = df.drop(['eu_o','eu_d'], axis=1)

In [14]:
#Fill in missing values as these were all between China/Taiwan/Hong Kong
df['heg_o'] = df['heg_o'].fillna(0)
df['heg_d'] = df['heg_d'].fillna(0)
df['col_dep_ever'] = df['col_dep_ever'].fillna(0)
df['sibling_ever'] = df['sibling_ever'].fillna(0)

In [15]:
#Export cleaned data
df.to_csv('cleaned.csv', index=False)